In [1]:
import urllib3
import requests
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Polygon, Point, MultiPoint, MultiPolygon, MultiLineString
import shapely
import tqdm
import copy

In [11]:
cred/AST_dataset/object_detection/volume_estimation/usgs_tnm_api.py --tile_level_annotations_path "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//complete_dataset_/tile_level_annotations/tile_level_annotations.geojson" --dataset_name "Lidar Point Cloud (LPC)" --request_total_idx "total" --request_content_idx "items" --request_content_names_idx "title"

# api access
https://ers.cr.usgs.gov/profile/access <br>
https://apps.nationalmap.gov/tnmaccess/ <br>
http get in python <br>
https://realpython.com/python-requests/ <br>
https://www.w3schools.com/tags/ref_httpmethods.asp <br>
https://stackoverflow.com/questions/645312/what-is-the-quickest-way-to-http-get-in-python <br>

In [5]:
tnm_url = 'https://tnmaccess.nationalmap.gov/api/v1/products'

# read tile level AST data

In [7]:
tile_level_annotations_path = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//complete_dataset_/tile_level_annotations/tile_level_annotations.geojson"
tile_level_annotations = gpd.read_file(tile_level_annotations_path)
#tile_level_annotations.columns

# Lidar Data
select the file with the largest bounding box/that is newest
matches file type "LAS,LAZ"

In [12]:
lpc_complete_df = usgs_api(tile_level_annotations, "Lidar Point Cloud (LPC)", "total", "items", "title")

100%|██████████████████████████████████████████████████████████████████████████| 98169/98169 [9:29:25<00:00,  2.87it/s]


In [15]:
lpc_complete_df[0:10]

,tank_idxs,usgs_tnm_names,urls,geometry
0,0,USGS Lidar Point Cloud NY LongIsland-Z18 2014 ...,https://rockyweb.usgs.gov/vdelivery/Datasets/S...,"POLYGON ((-73.7468593845017 40.6168374245241, ..."
1,1,USGS Lidar Point Cloud NY LongIsland-Z18 2014 ...,https://rockyweb.usgs.gov/vdelivery/Datasets/S...,"POLYGON ((-73.7468593845017 40.6168374245241, ..."
2,2,USGS Lidar Point Cloud NY LongIsland-Z18 2014 ...,https://rockyweb.usgs.gov/vdelivery/Datasets/S...,"POLYGON ((-73.7468593845017 40.6168374245241, ..."
3,3,USGS Lidar Point Cloud NY LongIsland-Z18 2014 ...,https://rockyweb.usgs.gov/vdelivery/Datasets/S...,"POLYGON ((-73.7468593845017 40.6168374245241, ..."
4,4,USGS Lidar Point Cloud NY LongIsland-Z18 2014 ...,https://rockyweb.usgs.gov/vdelivery/Datasets/S...,"POLYGON ((-73.7468593845017 40.6168374245241, ..."
5,5,USGS Lidar Point Cloud NY LongIsland-Z18 2014 ...,https://rockyweb.usgs.gov/vdelivery/Datasets/S...,"POLYGON ((-73.7468593845017 40.6168374245241, ..."
6,6,USGS Lidar Point Cloud NY LongIsland-Z18 2014 ...,https://rockyweb.usgs.gov/vdelivery/Datasets/S...,"POLYGON ((-73.7468593845017 40.6168374245241, ..."
7,7,USGS Lidar Point Cloud NY LongIsland-Z18 2014 ...,https://rockyweb.usgs.gov/vdelivery/Datasets/S...,"POLYGON ((-73.7468593845017 40.6168374245241, ..."
8,8,USGS Lidar Point Cloud NY LongIsland-Z18 2014 ...,https://rockyweb.usgs.gov/vdelivery/Datasets/S...,"POLYGON ((-73.7468593845017 40.6168374245241, ..."
9,9,USGS Lidar Point Cloud NY LongIsland-Z18 2014 ...,https://rockyweb.usgs.gov/vdelivery/Datasets/S...,"POLYGON ((-73.7468593845017 40.6168374245241, ..."


In [30]:
    #identify img/xml that have been moved
    #img_paths_copy = copy.copy(img_paths)
    #xml_paths_copy = copy.copy(xml_paths)

[]

In [195]:
#for geometry_list


0it [00:00, ?it/s]


In [ ]:
#check if bbox is included in list of lpc

#request lpc

In [175]:
lpc_gdf = None
if lpc_gdf == None:
    lpc_gdf = gpd.GeoDataFrame({'lpc_name': lpc_newest.index[0], "lpc_url": lpc_url, 'geometry': lpc_polygon})
    return(gdf)  


SyntaxError: incomplete input (922950664.py, line 1)

# DEM Data
select the file with the largest bounding box/that is newest
matches file type "LAS,LAZ"

In [20]:
dem_1m_dataset = ["Digital Elevation Model (DEM) 1 meter"]
dem_3m_dataset = ["National Elevation Dataset (NED) 1/9 arc-second"] #why is the list not including both
dataset_str = ",".join(map(str, dataset)) 
payload = {'bbox': bbox_str, "datasets": dataset}
response = requests.get(url, payload)
print(response.url)

https://tnmaccess.nationalmap.gov/api/v1/products?bbox=-104.92070147938492%2C39.77659825409954%2C-104.9206454910845%2C39.776652353263664&datasets=Digital+Elevation+Model+%28DEM%29+1+meter&datasets=National+Elevation+Dataset+%28NED%29+1%2F9+arc-second


In [21]:
#We can check whether the request is sucessful0
##first using the status code
status_code = response.status_code # status code, shows whether the request was sucess (200 -> OK) or not (404 -> Not Found
if status_code == 200:
    print('Success!')
elif status_code == 404:
    print('Not Found.')
    
##then use the response return itself
if response: #may not have actually worked (not just a 200 return), but returns a workable response (204 -> no content in message body for example
    print('Success!')
else:
    print('An error has occurred.')

Success!
Success!


In [22]:
#https://realpython.com/python-requests/
#get the content from  request 
request_content = response.content #payload of the get request in bytes
#print(request_content)
response.encoding = 'utf-8' # Optional: requests infers this internally
request_test = response.text #converts to a strong using a character encoder (UTF-8)
#print(request_test)
response.json()

#response.headers['Content-Type']#

{'total': 1,
 'items': [{'title': 'USGS NED ned19_n40x00_w105x00_co_denvercity_2008 1/9 arc-second 2009 15 x 15 minute IMG',
   'moreInfo': 'This tile of the National Elevation Dataset (NED) is 1/9 arc-second resolution. The National Elevation Dataset (NED) serves the elevation layer of The National Map, and provides basic elevation information for earth science studies and mapping applications in the United States. Scientists and resource managers use NED data for global change research, hydrologic modeling, resource monitoring, mapping and visualization, and many other applications. The NED is an elevation dataset that consists of seamless layers and a high resolution layer. Each of these layers are composed of the best available raster elevation data of the conterminous United States, Alaska, Hawaii, territorial islands, Mexico and Canada. The NED is updated continually [...]',
   'sourceId': '5d0ae9c6e4b0e3d3116021b5',
   'sourceName': 'ScienceBase',
   'sourceOriginId': None,
   '

In [16]:
#https://realpython.com/python-requests/
#get the content from  request 
request_content = response.content #payload of the get request in bytes
#print(request_content)
response.encoding = 'utf-8' # Optional: requests infers this internally
request_test = response.text #converts to a strong using a character encoder (UTF-8)
#print(request_test)
response.json()
#response.headers['Content-Type']#

{'total': 2,
 'items': [{'title': 'USGS 1 Meter 13 x50y441 CO_DRCOG_2020_B20',
   'moreInfo': 'This is a tile of the standard one-meter resolution digital elevation model (DEM) produced through the 3D Elevation Program (3DEP) . The elevations in this DEM represent the topographic bare-earth surface. USGS standard one-meter DEMs are produced exclusively from high resolution light detection and ranging (lidar) source data of one-meter or higher resolution. One-meter DEM surfaces are seamless within collection projects, but, not necessarily seamless across projects. The spatial reference used for tiles of the one-meter DEM within the conterminous United States (CONUS) is Universal Transverse Mercator (UTM) in units of meters, and in conformance with the North American Datum of 1983 (NAD83). All bare earth elevation values are in [...]',
   'sourceId': '620de50ed34e6c7e83baa048',
   'sourceName': 'ScienceBase',
   'sourceOriginId': None,
   'sourceOriginName': 'gda',
   'metaUrl': 'https:/

In [ ]:
https://tnmaccess.nationalmap.gov/api/v1/products?bbox=-104.92070147938492,%20%2039.77659825409954,%20%20-104.9206454910845,%20%2039.776652353263664